# 事前準備

タイムアウト回避

In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,59999)

<IPython.core.display.Javascript object>

実行設定値<br>

In [ ]:
#クライアントID, 実行日
clientId = "fdc"
execute_date= "20230501"


Gドライブフォルダパス

In [ ]:
fldGdrive = '/content/drive/'
fldGdriveColabMyDrive = fldGdrive + 'MyDrive/'
fldGdriveColabMyDriveData = fldGdriveColabMyDrive + 'ColabNotebooks/data/'

#インプットフォルダ
fldGdriveColabMyDriveDataInput = fldGdriveColabMyDriveData + 'input/' + clientId + "_" + execute_date + '/'

#アウトプットフォルダ
fldGdriveColabMyDriveDataOutput = fldGdriveColabMyDriveData + 'output/' + clientId + "_" + execute_date + '/'

BigQueryに接続

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'bwing-230309'
location = 'asia-northeast1'

client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

**BigQueryからQuery結果をCSV化** <br>
必要数だけ回す必要あり（１０００件単位のインプットCSVファイルを生成させる）

In [ ]:
start_row = 1001
end_row = 2000
qr = client.query("SELECT * FROM `bwing-230309.attachment_fdc.tf_parse_description_4_tokenized`(1001, 2000);")
df = qr.to_dataframe()
df.to_csv(fldGdriveColabMyDriveDataInput + "t5_1001-2000.csv", index = False)

インプットファイル名を配列に指定

In [ ]:
#Gドライブのインプットフォルダからファイル名を調査
import glob
files = glob.glob(fldGdriveColabMyDriveDataInput + "*")
for file in files:
    print(file)

/content/drive/MyDrive/ColabNotebooks/data/input/fdc_20230501/t5_1-1000.csv
/content/drive/MyDrive/ColabNotebooks/data/input/fdc_20230501/t5_1001-2000.csv
/content/drive/MyDrive/ColabNotebooks/data/input/fdc_20230501/stop_words


In [ ]:
#インプットファイルパス（配列）を指定（上記からコピペ）
input_file_names = [
"t5_1-1000.csv",
"t5_1001-2000.csv",
]

Gドライブ接続

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


アウトプットフォルダを作成

In [ ]:
import os

#インプットプットフォルダを作成する
#if not os.path.exists(fldGdriveColabMyDriveDataInput):
#  os.mkdir(fldGdriveColabMyDriveDataInput)

#アウトプットフォルダを作成する
if not os.path.exists(fldGdriveColabMyDriveDataOutput):
  os.mkdir(fldGdriveColabMyDriveDataOutput)

python Vチェック　pip 更新

In [ ]:
#!pip list
!python -V
!pip check
!pip list | grep numpy
!pip install -U numba

Python 3.10.11
ipython 7.34.0 requires jedi, which is not installed.
pygobject 3.36.0 requires pycairo, which is not installed.
numpy                         1.22.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install wheel setuptools pip --upgrade
!pip3 install wheel setuptools pip --upgrade

# 👇️ if you don't have pip in your PATH environment variable
!python -m pip install wheel setuptools pip --upgrade
!python3 -m pip install wheel setuptools pip --upgrade
!py -m pip install wheel setuptools pip --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: py: command not found


torch, transformersのインストール

In [ ]:
#!pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2 torchaudio==0.7.2

#!pip install -q transformers==4.4.2 sentencepiece
#!pip install -q transformers==4.11.3 sentencepiece

!pip install -qU torch==1.13.* torchtext==0.14.* torchvision==0.14.* torchaudio==0.13.* torchdata==0.5.1 torchmetrics==0.11.* \
    transformers==4.26.1 pytorch_lightning==1.9.3 sentencepiece==0.1.97

学習モデルを指定

In [ ]:
# 転移学習済みモデル
MODEL_DIR = "sonoisa/t5-base-japanese-title-generation"

# データインプット

入力文字列の正規化関数を定義

In [ ]:
#文字列の正規化の定義
#表記揺れを減らします。今回はneologdの正規化処理を一部改変したものを利用します。 処理の詳細はリンク先を参照してください。
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

データ抽出関数を定義

In [ ]:

import tarfile
import re

#抽出データ格納箱
all_data = []

#
def remove_brackets(text):
    text = re.sub(r"(^【[^】]*】)|(【[^】]*】$)", "", text)
    return text

#
def normalize_text(text):
    #assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

#
def read_id_name_desc(line):
    line = normalize_text(remove_brackets(line))
    row = line.split(',')
    id = row[0]
    name = row[1]
    desc = row[2]
    return id, name, desc

#
def read_file(input_file_path):               
  with open(input_file_path) as f:
    for index, line in enumerate(f):                  
        id, name, desc = read_id_name_desc(line)
        print(id, name, desc[0:20] + '...')

        if len(id) > 0 and len(name) > 0:
            all_data.append({
                "id": id,
                "name": name,
                "descrption": desc
                })

データをファイルから抽出し、チェック<br>
※スキップしてもよい（データに問題がないと判断するならば）

In [ ]:
for f in input_file_names:
  path = fldGdriveColabMyDriveDataInput + f
  read_file(path)
print(all_data)

# 推論に必要なクラス等の定義
推論にはPyTorch/Transformersを利用します。

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    
    T5ForConditionalGeneration,
    T5Tokenizer,

)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()
print(USE_GPU)

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir=fldGdriveColabMyDriveData,  # データセットのディレクトリ
    model_name_or_path=MODEL_DIR,
    tokenizer_name_or_path=MODEL_DIR,

    max_input_length=512,
    max_target_length=64,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)

True


# 学習済みモデルの読み込み

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

# 文章に対するタイトル生成

文章に合うタイトルを10個、自動生成します。
以下のコードではタイトルの多様性を生むために色々generateメソッドのパラメータを設定しています。パラメータの詳細は下記リンク先を参照してください。

generateメソッドのパラメータの意味
要約をGenerateさせたい文章ファイルを読み込み<br>
ループ処理でファイルに生成された文章ｊを書き出していく<br>
<br>
形式：{ID}, {Name}, {要約文}, {説明文}

In [ ]:
%cd input/
%ls
%pwd

[Errno 2] No such file or directory: 'input/'
/content
drive/  sample_data/


'/content'

In [ ]:
MAX_SOURCE_LENGTH = args_dict["max_input_length"]   # 入力される記事本文の最大トークン数
MAX_TARGET_LENGTH = args_dict["max_target_length"]  # 生成されるタイトルの最大トークン数

def preprocess_body(text):
    return normalize_text(text.replace("\n", " "))

#
def read_id_name_desc_line(line):
    #line = normalize_text(remove_brackets(line))
    row = line.split(',')
    id = row[0]
    name = row[1]
    desc = row[2].replace('【商品詳細】', '')
    return id, name, desc

#
def inference_summary(input_file_path, output_file_path):

  # 推論モード設定
  trained_model.eval()

  with open(input_file_path) as f:
    inferenceis = []
    for index, line in enumerate(f):
      id, name, desc = read_id_name_desc_line(line)
      #if index == 0:
      #  continue

      # 前処理とトークナイズを行う
      inputs = [preprocess_body(desc)]
      batch = tokenizer.batch_encode_plus(
          inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, 
          padding="longest", return_tensors="pt")
      input_ids = batch['input_ids']
      input_mask = batch['attention_mask']
      if USE_GPU:
          input_ids = input_ids.cuda()
          input_mask = input_mask.cuda()

      # 生成処理を行う
      outputs = trained_model.generate(
          input_ids=input_ids, attention_mask=input_mask, 
          max_length=MAX_TARGET_LENGTH,
          temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
          num_beams=10,             # ビームサーチの探索幅
          diversity_penalty=1.0,    # 生成結果の多様性を生み出すためのペナルティ
          num_beam_groups=10,       # ビームサーチのグループ数
          num_return_sequences=10,  # 生成する文の数
          repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
      )
      #print(outputs)

      # 生成されたトークン列を文字列に変換する
      generated_summary = [tokenizer.decode(ids, skip_special_tokens=True, 
                                          clean_up_tokenization_spaces=False) 
                          for ids in outputs]

      """
      # 生成されたタイトルを表示する
      for i, title in enumerate(generated_summary):
          print(f"{i+1:2}. {title}")
      """

      #書き出す文字列をformat
      sum = generated_summary[0]
      s = "{0},{1},{2},{3},{4}".format(index+1, id, name, sum.replace('|', ''), desc.replace('\n', ''))
      inferenceis.append(s)
      print(s)
      #if index == 3:
      #  break
      
  with open(output_file_path, mode='w') as f:
    for i, s in enumerate(inferenceis):
      #print(s)
      f.write(s+'\n')


## 要約を実行

*   入力ファイル名と出力先指定を
*   inference_summaryを実行(ファイルごとにループ上でCall)



In [ ]:
output_file_names = []
for f in input_file_names:
  input_file_path = fldGdriveColabMyDriveDataInput + f
  output_file_path = fldGdriveColabMyDriveDataOutput + "inferenced_t5_" + f
  output_file_names.append("inferenced_t5_" + f)
  inference_summary(input_file_path, output_file_path)

"""
#出力された結果ファイルの行数を出力する
import subprocess
for f in output_file_names:
  file = fldGdriveColabMyDriveDataInput + f
  cmd = "wc -l " + file
  row = subprocess.check_output(cmd.split()).decode().split()[0]
  s = "{0}:{1}".format(f, row)
  print(s)
"""

/usr/local/lib/python3.10/dist-packages/transformers/generation/beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


1,id,name,javascriptを有効にするには?,description
2,112049110001,ジュエリークリーナーセット,大切なジュエリーのお手入れにご使用下さい,大切なジュエリーの日々のお手入れにご使用下さい。※使用方法・使用上の注意をよく読み、正しくご使用ください。【セット内容】・ジュエリークリーナー（液につけるタイプのジュエリー専用クリーナー）・洗浄用容器・ブラシ・ピンセット・ポリッシングクロス（イエロー：ゴールド専用、ブルー：シルバー専用）・シルバーポリッシュ（シルバー製品の細かな傷を目立たなくし、変色を防止します）
3,152144050001,シルバー イヤーカフ/片耳用,手軽に楽しめるイヤージュエリー〈earjoy〉,【手軽に楽しめるイヤージュエリー〈earjoy〉】2連風のシルエットがトレンド感のある、イエローゴールドカラーの片耳用イヤーカフ。2本のデザインチェーンが耳元で光を受けてきらめきます。身に着けやすいサイズ感でどのような服装にも合う、1つ持っていると旬な耳元にしてくれるアイテムです。※こちらのシリーズは、一部店舗でのお取り扱いとなります。詳細は『取扱い店舗一覧』をご確認ください。
4,152144050004,シルバー イヤーカフ/片耳用,遊び心をプラスする、大人のイヤージュエリー〈earjoy〉,【手軽に楽しめるイヤージュエリー〈earjoy〉】ラインが連なったシルエットがトレンド感のある、イエローゴールドカラーの片耳用イヤーカフ。動きのあるラインがきらめき耳元に遊び心をプラスします。身に着けやすいサイズ感で、ピアスとのレイヤードも楽しめます。※こちらのシリーズは、一部店舗でのお取り扱いとなります。詳細は『取扱い店舗一覧』をご確認ください。
5,152144050005,シルバー イヤーカフ/片耳用,トレンド感のある地金カラーの耳用イヤーカフ【earjoy】,【手軽に楽しめるイヤージュエリー〈earjoy〉】トレンド感のある地金のラインが女性らしさを引き立てる、イエローゴールドカラーの片耳用イヤーカフ。動きのあるラインが耳を包み込み旬な耳元を演出します。身に着けやすいサイズ感で、どのような服装にも合うアイテムです。※こちらのシリーズは、一部店舗でのお取り扱いとなります。詳細は『取扱い店舗一覧』をご確認ください。
6,1

'\n#出力された結果ファイルの行数を出力する\nimport subprocess\nfor f in output_file_names:\n  file = fldGdriveColabMyDriveDataInput + f\n  cmd = "wc -l " + file\n  row = subprocess.check_output(cmd.split()).decode().split()[0]\n  s = "{0}:{1}".format(f, row)\n  print(s)\n'